In [2]:
%pip install python-dotenv
from  dotenv import load_dotenv
load_dotenv()
import os

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pinecone-client
%pip install openai

  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_inference-1.0.3-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Using cached pinecone_plugin_inference-1.0.3-py3-none-any.whl (117 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx

In [30]:
from openai import OpenAI
from pinecone import Pinecone,ServerlessSpec

In [31]:
pc= Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(name="rag",dimension=1536,metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1"))

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '9fe0cd76e4a2d4bd65ff6af8d7f74329', 'Date': 'Thu, 22 Aug 2024 01:17:33 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [18]:
import json
data=json.load(open("reviews.json"))


In [37]:
process_data=[]
client = OpenAI()

for  review in data['reviews']:
    response=client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"

    )
    embedding=response.data[0].embedding
    process_data.append({
        "values":embedding,
        "id":review['professor'],
        "metadata":{
            "review":review['review'],
            "subject":review['subject'],
            "stars":review['stars'],
        }

   } )


In [38]:
index=pc.Index('rag')
index.upsert(
    vectors=process_data,
    namespace="ns1"
)

{'upserted_count': 30}

In [39]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 30}},
 'total_vector_count': 30}